<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Line-intersection" data-toc-modified-id="Line-intersection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Line intersection</a></span><ul class="toc-item"><li><span><a href="#Combined-version" data-toc-modified-id="Combined-version-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Combined version</a></span></li><li><span><a href="#Examples" data-toc-modified-id="Examples-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Examples</a></span></li></ul></li></ul></div>

# CUDA 2D Raycaster for Line Maps with Segment IDs

In [1]:
display(HTML("<style>.rendered_html.text_cell_render {max-width:500px; }</style>"));

HTML{String}("<style>.rendered_html.text_cell_render {max-width:500px; }</style>")

In [2]:
#nbx --fname="src/cuda_raycaster_line_map_c.jl"
push!(LOAD_PATH, "src");
using MyUtils
using CUDA
using MyCudaUtils
using BenchmarkTools
using Colors, Plots
col = palette(:default);
import Base: @doc

[ Info: Precompiling MyUtils [top-level]


In [3]:
include("./src/pose.jl");
using Geometry: Segment

[ Info: Precompiling Geometry [top-level]


## Line intersection

To do raycsting we need to compute the intersection of two lines by solving: $x + s \cdot dx = y + t \cdot dy$.

Implementation from my `Geometry.jl` package:

```julia
function line_intersect(x, x′, y, y′)
    dx = x′ - x
    dy = y′ - y
    v  = x - y
    if det([-dx dy]) == 0
        return [Inf;Inf]
    end
    s, t = inv([-dx dy])*v
    return s,t
end;
```

And here's a version without arrays that we can use within a cuda kernel later.

In [8]:
function intersections(x, x′, y, y′)
    n = ndims(x)
    dx = x′ .- x
    dy = y′ .- y
    v  = x .- y

    dx1 = selectdim(dx, n, 1)
    dx2 = selectdim(dx, n, 2)
    dy1 = selectdim(dy, n, 1)
    dy2 = selectdim(dy, n, 2)
    v1  = selectdim(v, n, 1)
    v2  = selectdim(v, n, 2)

    a, b = -dx1, dy1
    c, d = -dx2, dy2

    det = a.*d .- b.*c
    
    s = 1 ./det .*(  d.*v1 .- b.*v2)
    t = 1 ./det .*(- c.*v1 .+ a.*v2)
    return s,t, x .+ s.*dx, y .+ t.*dy
end

intersections (generic function with 1 method)

In [9]:
create_angles(fov, num_a) = [range(-fov/2, fov/2, num_a)...];

In [31]:
function cast(v0::AbstractVector, as::AbstractVector, segs::AbstractArray, zmax=Inf)
    x  = view(v0, 1:2)
    hd = view(v0, 3)
    as = as .+ hd

    x′ = reshape(x,1,2) .+ cat(cos.(as), sin.(as), dims=2)
    y  = view(segs,:,1:2)
    y′ = view(segs,:,3:4)

    x  = reshape(x , :, 1, 2)
    x′ = reshape(x′, :, 1, 2)
    y  = reshape(y , 1, :, 2)
    y′ = reshape(y′, 1, :, 2)

    s, t, _, _ = intersections(x, x′, y, y′)

    # Hit map
    h = (0 .< s) .* (0 .<= t .<= 1)
    s[.!h] .= zmax

    # Segment-ID
    i  = argmin(s, dims=2)[:,1]
    i2 = map(i->i[2],i);
    
    # Todo: There was an issue when I computed `z_ = s_[i_]`
    #       Not sure why... but using minimum() works
    z = minimum(s, dims=2)[:,1]
    i2[z.==zmax] .= 0

    # Depth and Segment-ID
    return z, i2
end;

In [32]:
function cast(vs::AbstractArray, as::AbstractVector, segs::AbstractArray, zmax=Inf)
    map(v->cast(v, as, segs, zmax), eachslice(vs, dims=1))
end;

In [40]:
_fov   = π
_num_a = 100
zmax=100.0

vs   = rand(10,3)
segs = 5*(rand(40,4))
as   = create_angles(_fov, _num_a)

@btime cast($vs, $as, $segs) samples=1 evals=5;


  2.005 ms (1065 allocations: 2.65 MiB)
